# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

In [2]:
install.packages("tokenizers")
library(httr)
library(tokenizers)
library(stringr)

also installing the dependency 'SnowballC'




package 'SnowballC' successfully unpacked and MD5 sums checked
package 'tokenizers' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Ella Sharpe\AppData\Local\Temp\RtmpKKj4IV\downloaded_packages


Warning message:
"package 'tokenizers' was built under R version 4.5.2"


#### a) Make a function to tokenize the text.

In [3]:
tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase=TRUE, strip_punct=TRUE)[[1]]
}

#### b) Make a function generate keys for ngrams.

In [4]:
key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse=sep)
}


#### c) Make a function to build an ngram table.

In [5]:
build_ngram_table <- function(tokens, n, sep = "\x1f") {
    if (length(tokens) < n) return(new.env(parent = emptyenv()))
    tbl <- new.env(parent = emptyenv())
    for (i in seq_len(length(tokens) - n + 1L)) {
        ngram <- tokens[i:(i + n - 2L)]
        next_word <- tokens[i + n - 1L]
        key <- paste(ngram, collapse = sep)
        counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
        if (next_word %in% names(counts)) {
            counts[[next_word]] <- counts[[next_word]] + 1L
        } else {
            counts[[next_word]] <- 1L
        }
        tbl[[key]] <- counts
    }
    tbl
}

#### d) Function to digest the text.

In [6]:
digest_text <- function(text, n) {
    tokens <- tokenize_text(text)
    build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [7]:
digest_url <- function(url, n) {
    res <- httr::GET(url)
    txt <- httr::content(res, as = "text", encoding = "UTF-8")
    digest_text(txt,n)
}

#### f) Function that gives random start.

In [8]:
random_start <- function(tbl, sep = "\x1f") {
    keys <- ls(envir = tbl, all.names=TRUE)
    if (length(keys)==0) stop("No n-grams available. Digest text first.")
    picked <- sample(keys, 1)
    strsplit(picked, sep, fixed=TRUE)[[1]]
}

#### g) Function to predict the next word.

In [9]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
    key <- paste(ngram, collapse = sep)
    counts <- if(!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (length(counts) == 0) return(NA_character_)
    sample(names(counts), size=1, prob=as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [10]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl); n <- as.integer(n); force(sep)
    function(start_words = NULL, length = 10L) {
        if ((is.null(start_words)) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep=sep)
        }
        word_sequence <- start_words
        for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep=sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }
        paste(word_sequence, collapse= " ")
    }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 

In [16]:
set.seed(2025)
url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"
tbl3 <- digest_url(url, n=3)
gen3 <- make_ngram_generator(tbl3, n=3)

output <- gen3(start_words = c("the", "king"), length = 15)

cat(output)

the king has forbidden me to marry another husband am not i shall ride upon

#### ii) Using n=3, with no start word, with length=15.

In [18]:
url <- "https://www.gutenberg.org/cache/epub/10662/pg10662.txt"
tbl <- digest_url(url, n=3)
gen <- make_ngram_generator(tbl, n=3)

print(gen(length=15))

[1] "and circumstances which beset us about waiting until the heavier vibrations were come no farther"


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 

In [19]:
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"
tbl1 <- digest_url(url, n=3)
gen1 <- make_ngram_generator(tbl1, n=3)

output <- gen1(start_words = c("the", "king"), length = 15)

cat(output)

the king is also defective you may demand a refund of any kind express or

#### ii) Using n=3, with no start word, with length=15.

In [20]:
url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"
tbl2 <- digest_url(url, n=3)
gen2 <- make_ngram_generator(tbl2, n=3)

print(gen(length=15))

[1] "utter warm and dry the bandages that she saw me not greatly nor to breathe"


#### c) Explain in 1-2 sentences the difference in content generated from each source.

The content generated from Grimm's Fairy Tales seems to have more flowery wording than that of the Ancient Armour and Weapons in Europe. 

## Question 3
#### a) What is a language learning model? 

A language learning model is a model based on some body of text that uses that text to predict the next word in a sequence. In short, it is a probability distribution over words. 

#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

To run a language model locally, you can download Ollama to run an open access language model. 

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | The shell around the OS allows you to interact with all functions of the system, such as interpreting commands like mkdir project |
| **Terminal emulator** | The terminal emulator is what hosts the shell and allows you to interact with it: it provides the interface where you type mkdir project. |
| **Process** | Something running on the computer, such as the mkdir program. |
| **Signal** | Things to send to processes for them to do something |
| **Standard input** | Reads characters from the input |
| **Standard output** | Writes the normal output for a program|
| **Command line argument** | Things that we can pass to a process when we start it|
| **The environment** | All of the things a process can see when it is running |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?

The programs are find, xargs, and grep. 

#### b) Explain what this command is doing, part by part.

This command is finding everywhere a particular function is mentioned in a project. Find is searching for files that end in ".R." Then, the pipe operator is connecting find to xargs which allows us to search each individual file with grep. Grep allows the command to search for things in the files. 

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions. 
#### a) Show the response when you run `docker run hello-world`.

$ docker run hello-world
Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
17eec7bbc9d7: Pull complete
Digest: sha256:56433a6be3fda188089fb548eae3d91df3ed0d6589f7c2656121b911198df065
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/

#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.

$ docker run -it -p 8787:8787 rocker/verse
[s6-init] making user provided files available at /var/run/s6/etc...exited 0.
[s6-init] ensuring user provided files have correct perms...exited 0.
[fix-attrs.d] applying ownership & permissions fixes...
[fix-attrs.d] done.
[cont-init.d] executing container initialization scripts...
[cont-init.d] 01_set_env: executing...
skipping /var/run/s6/container_environment/HOME
skipping /var/run/s6/container_environment/RSTUDIO_VERSION
[cont-init.d] 01_set_env: exited 0.
[cont-init.d] 02_userconf: executing...


The password is set to phai6mah2pheTh8f
If you want to set your own password, set the PASSWORD environment variable. e.g. run with:
docker run -e PASSWORD=<YOUR_PASS> -p 8787:8787 rocker/rstudio


[cont-init.d] 02_userconf: exited 0.
[cont-init.d] done.
[services.d] starting services
[services.d] done.
TTY detected. Printing informational message about logging configuration. Logging configuration loaded from '/etc/rstudio/logging.conf'. Logging to 'syslog'.
^C[cont-finish.d] executing container finish scripts...
[cont-finish.d] done.
[s6-finish] waiting for services.
s6-svwait: fatal: timed out
[s6-finish] sending all processes the TERM signal.
TTY detected. Printing informational message about logging configuration. Logging configuration loaded from '/etc/rstudio/logging.conf'. Logging to 'syslog'.
malloc_consolidate(): unaligned fastbin chunk detected
[s6-finish] sending all processes the KILL signal and exiting.

#### c) How do you log in to the RStudio server?

You log into the RStudio server by going to http://localhost:8787 and entering the username (rstudio) and given password (phai6mah2pheTh8f)